# Evolución de Prompts — Preentrega 2

Este notebook muestra cómo se fue optimizando el prompt paso a paso (Fast Prompting).
Cada bloque incluye:
- Una explicación en **Markdown**.
- Una celda de código ejecutable de manera independiente.

Los 5 niveles son:
1. Prompt básico (sin optimización).
2. Prompt intermedio.
3. Prompt avanzado (JSON inicial).
4. Prompt actual (optimizado).
5. Prompt ultra optimizado (nivel propuesto).


In [ ]:
from openai import OpenAI
import os, json
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Datos base
destino = "Mendoza"
cant_dias = 3
cant_personas = 2
presupuesto = "200000"
moneda = "ARS"
intereses = "Deportes Invernales"
modo_viaje = "Exprímelo"
fecha_tentativa = "20/7/2025"

def safe_json_parse(raw_text: str):
    cleaned = raw_text.strip()
    if cleaned.startswith("```"):
        cleaned = cleaned.strip("`")
        cleaned = cleaned.replace("json\n", "").replace("json", "", 1).strip()
    try:
        return json.loads(cleaned)
    except Exception:
        print("⚠️ Error al parsear JSON:\n", cleaned)
        return {}


## Prompt 1 — Básico
No aplica técnicas de Fast Prompting, simplemente pide un itinerario genérico.

In [ ]:
prompt1 = f"""
Organizá un viaje de {cant_dias} días a {destino}.
Incluí actividades, comidas y presupuesto.
"""
resp1 = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user","content":prompt1}])
print(resp1.choices[0].message.content)

## Prompt 2 — Intermedio
Introduce un rol (organizador) y pide itinerario día a día, pero sin validaciones ni JSON.

In [ ]:
prompt2 = f"""
Actuá como organizador de escapadas. 
Con estos datos: destino={destino}, días={cant_dias}, personas={cant_personas}, presupuesto={presupuesto} {moneda}, intereses={intereses}.
Armá un itinerario día por día con horarios y comidas.
Incluí un presupuesto dividido en transporte, alojamiento y actividades.
"""
resp2 = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user","content":prompt2}])
print(resp2.choices[0].message.content)

## Prompt 3 — Avanzado
Introduce validación y supuestos explícitos. Pide salida en JSON simple.

In [ ]:
prompt3 = f"""
Actuá como organizador de escapadas de 2–3 días.
Datos: destino={destino}, días={cant_dias}, personas={cant_personas}, presupuesto={presupuesto} {moneda}, intereses={intereses}, modo={modo_viaje}.
Si falta algo, agregalo en 'supuestos'.
Devolvé SOLO un JSON con parámetros, supuestos y presupuesto general.
"""
resp3 = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user","content":prompt3}])
print(resp3.choices[0].message.content)

## Prompt 4 — Optimizado
Este prompt incluye normalización, supuestos explícitos, criterios de planificación, salida JSON y Markdown, además del cálculo de tokens.

In [ ]:
intake_prompt4 = f"""
Actuá como un organizador de escapadas de 2–3 días.
Normalizá la información del usuario y devolvé SOLO un JSON con parámetros, supuestos, criterios y alertas.
"""
resp4 = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user","content":intake_prompt4}])
print(resp4.choices[0].message.content)

## Prompt 5 — Ultra optimizado
Incluye campo 'ajustes', QA automático y agrupación geográfica. Salida estricta en JSON + itinerario detallado.

In [ ]:
intake_prompt5 = f"""
Sos un organizador experto en escapadas 2–3 días. 
Validá todos los campos, completá 'supuestos', corregí en 'ajustes', definí criterios y QA. 
Devolvé SOLO JSON con parámetros, supuestos, ajustes, criterios, presupuesto, checklist, alertas y qa.
"""
resp5 = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user","content":intake_prompt5}])
print(resp5.choices[0].message.content)